1. Enter usernames to scrap
2. Get their data
3. Filter based on text data
4. Remove those images
5. Filter based on GCP
6. Remove those images
7. Add one column with Food or not(if you want food specific photos)
8. Caption for each post and shuffle df
9. Create a copy of df and Post Daily using df
10. Manualy curated Comments for interaction(can be changed according to your needs)
11. Interact with other accounts and get more followers

You will notice cells showing error after running, these are not real, i removed my personal info and ran cells again

In [1]:
!pip install ffmpeg
!pip install instagram-scraper
!pip install google-cloud-vision
!pip install instapy
!pip install pandas


  Using cached https://files.pythonhosted.org/packages/0a/0e/ea53a3d6f1eb2cc31162c9ae89555cc26a3986e5559781f0b0df75aea5cf/tqdm-4.50.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/45/1e/0c169c6a5381e241ba7404532c16a21d86ab872c9bed8bdcd4c423954103/requests-2.24.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ed/1b/72a1821152d07cf1d8b6fce298aeb06a7eb90f4d6d41acec9861e7cc6df0/decorator-4.4.2-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/b8/e5/a64ef44a85397ba3c377f6be9c02f3cb3e18023f8c89850dd319e7945521/numpy-1.19.2-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/6e/57/5d899fae74c1752f52869b613a8210a2480e1a69688e65df6cb26117d45d/imageio-2.9.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/bc/a9/01ffebfb562e4274b6487b4bb1ddec7ca55ec7510b22e4c51f14098443b8/chardet-3.0.4-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/pac

In [1]:
#might be needed in some cases
import subprocess,os
import os
os.environ["IMAGEIO_FFMPEG_EXE"] = "/usr/bin/ffmpeg"

In [5]:
#scrap data from instagram accounts provided in ig_users.txt file
#For more information on parameters and other parameters you can check out this github 
#link:https://github.com/arc298/instagram-scraper

with open('ig_users.txt','r') as f:
    usernames = f.readlines()
    print(usernames)
    for user in usernames:
        subprocess.run(args= ["instagram-scraper", "{}".format(user),  "-t",\
                              "video", "--latest-stamps", "timestamps.txt", "-q", "--media-metadata", "-d", "page_data/{}".format(user.strip('\n'))])
    

[]


In [85]:
#creating a dataframe from multiple json files present in multiple scraped folder with info about each image/video
import pandas as pd
import os,json
df = pd.DataFrame()
pwd = os.path.join(os.getcwd(),"page_data")
for folder in os.listdir(pwd):
    x = os.path.join(pwd,folder)
    pwd_json = os.listdir(x)
    for item in pwd_json:
        if item.endswith(".json"):
            f = open(os.path.join(x,item),encoding="utf8") 
            data = json.load(f) 
            f.close()
            df1 = pd.DataFrame.from_dict(pd.json_normalize(data['GraphImages']), orient='columns')
            df = df.append(df1,ignore_index=True)
            
print(df.shape)       
    

(16271, 21)


In [86]:
#Processing dataframe for removing posts with 
# 1. disabled comments
# 2. likes< 100 and comments<10

df = df[df.comments_disabled  == False]

df = df.sort_values(by = 'taken_at_timestamp')

df = df[df.taken_at_timestamp < 1584921600] #before coronavirus

df = df[df['tags'].notna()]

df = df[df['edge_media_preview_like.count']>100]

df = df[df['edge_media_to_comment.count']>10]

df['edge_media_to_caption.edges'] = df['edge_media_to_caption.edges'].apply(lambda x : x[0]['node']['text'])

df['edge_media_to_caption.edges'] = df['edge_media_to_caption.edges'].apply(lambda x : x.lower())



In [87]:
# remove a post if it's caption contains words like buy link limited etc(you can add more)
df = df[~df['edge_media_to_caption.edges'].str.contains("buy|link|limited", na=False)] 

In [88]:
#I don't want videos for now
df = df[df.is_video!=True]

In [89]:
df.shape

(1337, 21)

In [93]:
#Creating a new tag column which contains name of the user who i want to give credit for the image.
# Observation : many posts of a user give credits to other users if they give, i use those or i give credit to the user
df['new_tag'] = df[df['edge_media_to_caption.edges'].str.contains\
                 ("picture credit|photography by|shot by|capture by")]['edge_media_to_caption.edges'].\
apply(lambda x: [i for i in x.split() if i[0] == '@' or i[:2] == '_@' ])
#df_temp = df[df.new_tag.notna()]
#df_temp.dropna(subset = ['new_tag'], inplace=True)
df['new_tag'].fillna('@'+df.username, inplace = True)

In [103]:
#extracting image name 
import re
df['image_name'] = df.urls.apply(lambda x: re.search(r'/.*/(.*)\.mp4', x[0]).group(1) + ".mp4")
#df.display_url.apply(lambda x: re.search(r'/e.*/(.*)\.jpg', x).group(1) + ".jpg")

In [105]:
df.reset_index(inplace=True)

In [106]:
df.shape

(1337, 24)

In [107]:
#removing duplicate images if any(no chance but still for safety)
df[df.duplicated(subset=['image_name'])==True]

,index,__typename,comments_disabled,display_url,gating_info,id,is_video,media_preview,shortcode,tags,...,username,dimensions.height,dimensions.width,edge_media_preview_like.count,edge_media_to_caption.edges,edge_media_to_comment.count,owner.id,video_view_count,new_tag,image_name


In [6]:
# creating a new folder images,which has images which are present in our final dataframe(processed df)
import pandas as pd
import os,json
import shutil

dst_dir = '/home/johncena/Videos/insta/images'  #change directory name

pwd = os.path.join(os.getcwd(),"page_data")
for folder in os.listdir(pwd):
    x = os.path.join(pwd,folder)
    print(x)
    pwd_json = os.listdir(x)
    for item in pwd_json:
        if item.endswith(".jpg") and item in df.image_name.tolist():
            shutil.copy(x+"/"+item,dst_dir)

/home/johncena/Videos/insta/page_data/sodelhi


NameError: name 'df' is not defined

In [110]:
src_dir = '/home/johncena/Videos/insta/images'  #Change accordingly

images= []
for image in os.listdir(src_dir):
    images.append(image)

new_df = df[df.image_name.isin(images)] # To prevent errors

#Final DF
new_df.shape

You can continue without google cloud api, but make changes to df names accordingly at the last section of this notebook.

### Google cloud processing

In [348]:
# Using gcp for doing image analysis for detecting images with watermarks, objects present in image to provide a object specific caption.
# if an image contains food related items then provide a foodie caption.
#Usage of the results can be tweaked according to your needs.


from google.cloud import vision
#Downloaded JSON FIle from google cloud platform
client = vision.ImageAnnotatorClient.from_service_account_json('/home/johncena/Videos/insta/your_service_key.json')

text = []
data =[]
obj = []

src_dir = '/home/johncena/Videos/insta/images'


for image in os.listdir(src_dir):
    with open(os.path.join(src_dir,image), 'rb') as image_file:
        content = image_file.read()
    response = client.text_detection({'content':content,})
    response1 = client.object_localization({'content':content,})
    data.append(response.text_annotations)
    obj.append([response1.localized_object_annotations[i].name for i in range(len(response1.localized_object_annotations))])
    if len(response.text_annotations)> 0:
        text.append(1)
    else:
        text.append(0) 



ServiceUnavailable: 503 Socket closed

In [ ]:
new_df['text'] = text

In [112]:
new_df['obj'] = obj

/snap/jupyter/6/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [409]:
new_df_no_text = new_df[new_df['text']==0] # Removing all rows with images having a text inside them(optical character recognition)

Food or not Logic based column Food

In [411]:
food = []
for i, row in df_1_no_text.iterrows():
    if 'Food' in ' '.join(row['tags']).lower() + " " + ' '.join(row['obj']).lower():
        food.append(1)
    else:
        food.append(0)

In [413]:
new_df_no_text['Food'] = food

/snap/jupyter/6/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [147]:
f_hash = ["#food","#foodporn","#foodie","#instafood","#foodphotography","#foodstagram","#yummy","#instagood"]

In [13]:
o_hash = ["#other","#okay"]

In [148]:
food = ["The only thing I like better than talking about Food is eating.",
"If food were free, why work?",
"Your diet is a bank account. Good food choices are good investments.",
"Food is for eating, and good food is to be enjoyed… I think food is, actually, very beautiful in itself.",
"There is no “we” in fries. Remember that!",
"There’s nothing more romantic than Italian food.",
"I’m on a 30-day diet. So far, I’ve lost 15 days.",
"Where there is good food, there is happiness.",
"Let’s eat some diet food while we wait for the steak to cook.",
"In the mood for noods.",
"Eat today, live another day.",
"I’m never giving up on chocolate! I’m not a quitter!",
"A day is best started on a full stomach."]


In [149]:
other = [
"I love It",
"It loves me and I love it",
"God created It with dil"]

In [2]:
#Assigning random number of hashtags and random caption for each row(post) for different types of objects.
# if an image has food, then assigning food based hashtags(different number for every post) and randomly chosen caption
# caption has caption + photo credits + hashtags
import random

cap = []
for i,row in df_no_text.iterrows():
    p = random.choice([13,14,15,16,17,19,])
    if row['Food'] == 1:
        cap.append(random.choice(food) + " \n" + "Photo credits: " + str(row['new_tag']) + " \n" +\
                   ' '.join(random.sample(f_hash,p)))
    else:
        cap.append(random.choice(delhi) + " \n" + "Photo credits: " + row['new_tag'] + " \n" +\
                   ' '.join(random.sample(d_hash,p)))

NameError: name 'df_no_text' is not defined

In [ ]:
new_df_no_text['caption'] = cap

In [178]:
final_df1 = new_df_no_text[['image_name','caption']]

In [179]:
#shuffling df for more randomness
final_df1 = final_df1.sample(frac=1).reset_index(drop=True)

In [3]:
final_df1

NameError: name 'final_df1' is not defined

In [181]:
final_df1.to_csv('/home/johncena/Videos/insta/page1',index=False, header=True)

In [182]:
#Copying images whicha re present in final images to new folder(final_images) which we will use for posting on instagram
new_images = final_df1.image_name.tolist()

dst_dir = '/home/johncena/Videos/insta/final_images'

pwd = os.path.join(os.getcwd(),"images")
for folder in os.listdir(pwd):
    if folder in final_df1.image_name.tolist():
        shutil.copy(pwd+"/"+folder,dst_dir)

1336

## Post content automatically and interact with other users to increase followers

You will find interact.py file, which will use post.py for posting automatically and interact with other accounts and garner more followers, i have provided you with the best settings.
Observations after running it in september 2020:
1. You will get more followers after you follow popular instagram accounts I have provided the list 
2. Unfollow accounts after 2 days
3. Use tags to search for your audience and interact with them and write comments and follow them (example: food, yummy etc if you are a food blogger)